In [ ]:
import numpy as np
import math

SR        = 16000
NFFT      = 512
MicNum    = 4
room_size = np.array([5.0, 5.0, 3.3])
c         = 343
r         = 0.065 / 2
Sor_dis   = [2.27055, 2.40018]
Sor_theta = [90, 270]
Sor_phi   = [18.75414, 21.38029]
DOA       = np.array( [Sor_theta, Sor_phi]).T # variable


def Sph2Cas2(angle):
    assert angle.ndim==2
    angle_tmp = angle.copy()*math.pi/180
    x = (np.cos(angle_tmp[:,0])*np.sin(angle_tmp[:,1]))[:,None]
    y = (np.sin(angle_tmp[:,0])*np.sin(angle_tmp[:,1]))[:,None]
    z = (np.cos(angle_tmp[:,1]))[:,None]
    return np.concatenate( (x, y, z), axis=1)


kappa           = Sph2Cas2(DOA) # variable
ff              = SR/NFFT * np.arange(0,int(NFFT//2)+1)   # (ff) fix
k               = 2*math.pi*ff / c                        # (ff) fix
dMicStrc        = np.array([[ 0., 0., 0.], [-0.0325, 0.0325, 0.], [-0.065, 0., 0.], [-0.0325, -0.0325, 0.]]) # fix
Af              = np.exp(1j*k[None, None, :]*((kappa @ dMicStrc.T)[:,:,None]))  # (sor, mic, ff), free field RTF

print(Af)



[[[ 1.        +0.00000000e+00j  1.        +0.00000000e+00j
    1.        +0.00000000e+00j ...  1.        +0.00000000e+00j
    1.        +0.00000000e+00j  1.        +0.00000000e+00j]
  [ 1.        +0.00000000e+00j  0.99998211+5.98147334e-03j
    0.99992844+1.19627327e-02j ...  0.05147028+9.98674526e-01j
    0.04549582+9.98964529e-01j  0.03951973+9.99218791e-01j]
  [ 1.        +0.00000000e+00j  1.        -7.32523587e-19j
    1.        -1.46504717e-18j ...  1.        -1.86060991e-16j
    1.        -1.86793515e-16j  1.        -1.87526038e-16j]
  [ 1.        +0.00000000e+00j  0.99998211-5.98147334e-03j
    0.99992844-1.19627327e-02j ...  0.05147028-9.98674526e-01j
    0.04549582-9.98964529e-01j  0.03951973-9.99218791e-01j]]

 [[ 1.        +0.00000000e+00j  1.        +0.00000000e+00j
    1.        +0.00000000e+00j ...  1.        +0.00000000e+00j
    1.        +0.00000000e+00j  1.        +0.00000000e+00j]
  [ 1.        +0.00000000e+00j  0.999977  -6.78235657e-03j
    0.999908  -1.35644011e-02

In [6]:
RTF = np.transpose(Af, (2, 1, 0))       # (freq, mic, sor)
f   = RTF.shape[0]
SorNum = RTF.shape[2]

pinvA = [np.linalg.inv(RTF[ii].conj().T @ RTF[ii] + 1e-2 * np.eye(SorNum)) @ RTF[ii].conj().T
        for ii in range(f)]

print(pinvA)


[array([[0.12484395+0.j, 0.12484395+0.j, 0.12484395+0.j, 0.12484395+0.j],
       [0.12484395+0.j, 0.12484395+0.j, 0.12484395+0.j, 0.12484395+0.j]]), array([[0.12484648-7.15124310e-17j, 0.1250954 -6.27911051e-01j,
        0.12484648+8.70103821e-17j, 0.1250954 +6.27911051e-01j],
       [0.12484648+7.15124310e-17j, 0.12459246+6.28011038e-01j,
        0.12484648-8.72300237e-17j, 0.12459246-6.28011038e-01j]]), array([[0.1248541 -1.35998152e-16j, 0.12580359-1.19817304e+00j,
        0.1248541 +1.66490658e-16j, 0.12580359+1.19817304e+00j],
       [0.1248541 +1.35998152e-16j, 0.1238842 +1.19837302e+00j,
        0.1248541 -1.66929968e-16j, 0.1238842 -1.19837302e+00j]]), array([[0.1248668 -1.67794469e-16j, 0.1268497 -1.66953087e+00j,
        0.1248668 +2.53686298e-16j, 0.1268497 +1.66953087e+00j],
       [0.1248668 +1.67794469e-16j, 0.12283795+1.66983084e+00j,
        0.1248668 -2.54345330e-16j, 0.12283795-1.66983084e+00j]]), array([[0.12488458-2.18695378e-16j, 0.12808707-2.02461332e+00j,
       